<a href="https://colab.research.google.com/github/keshavkrk/SDC/blob/main/YoutubeSummariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtube-transcript-api
!pip install langchain faiss-cpu
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 38.3 MB/s eta 0:00:00


In [3]:
!pip install -U langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

from transformers import pipeline


In [5]:
def get_transcript_from_youtube(video_id):
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    full_text = " ".join([t["text"] for t in transcript])
    return full_text


In [6]:
def create_vectorstore_from_text(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(text)

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)
    return vectorstore


In [7]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [8]:
summarization_llm = HuggingFacePipeline(pipeline=summarizer)


<ipython-input-8-70eb2b9a6145>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  summarization_llm = HuggingFacePipeline(pipeline=summarizer)


def build_rag_chain(vectorstore):
    qa_chain = RetrievalQA.from_chain_type(
        llm=summarization_llm,
        retriever=vectorstore.as_retriever(),
        return_source_documents=False
    )
    return qa_chain


In [9]:
def build_rag_chain(vectorstore):
    qa_chain = RetrievalQA.from_chain_type(
        llm=summarization_llm,
        retriever=vectorstore.as_retriever(),
        return_source_documents=False
    )
    return qa_chain


In [10]:
def summarize_youtube_video(video_id, query="Summarize this video"):
    transcript = get_transcript_from_youtube(video_id)
    vectorstore = create_vectorstore_from_text(transcript)
    rag_chain = build_rag_chain(vectorstore)
    result = rag_chain.run(query)
    return result


In [12]:
video_id = "ry9SYnV3svc"
summary = summarize_youtube_video(video_id)
print("🔍 Summary:\n", summary)


🔍 Summary:
 Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don'st know, don't try to make up an answer. Summarize this video to help people with reading comprehension and vocabulary.


In [13]:
!pip install youtube-transcript-api langchain langchain-community faiss-cpu transformers


In [14]:
# STEP 1: Import the required libraries
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, VideoUnavailable
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

# STEP 2: Extract transcript safely
def get_transcript_from_youtube(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = " ".join([t["text"] for t in transcript])
        return full_text
    except TranscriptsDisabled:
        return "❌ Transcripts are disabled for this video."
    except NoTranscriptFound:
        return "❌ No transcript found for this video."
    except VideoUnavailable:
        return "❌ The video is unavailable or private."
    except Exception as e:
        return f"❌ An unexpected error occurred: {str(e)}"

# STEP 3: Create Vectorstore from transcript
def create_vectorstore_from_text(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(text)

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)
    return vectorstore

# STEP 4: Build RAG chain
def build_rag_chain(vectorstore):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summarization_llm = HuggingFacePipeline(pipeline=summarizer)

    qa_chain = RetrievalQA.from_chain_type(
        llm=summarization_llm,
        retriever=vectorstore.as_retriever(),
        return_source_documents=False
    )
    return qa_chain

# STEP 5: Main summary function
def summarize_youtube_video(video_id, query="Summarize this video to help people with reading comprehension and vocabulary."):
    transcript = get_transcript_from_youtube(video_id)
    if transcript.startswith("❌"):
        return transcript  # Return error message

    vectorstore = create_vectorstore_from_text(transcript)
    rag_chain = build_rag_chain(vectorstore)
    result = rag_chain.run(query)
    return result

# STEP 6: Provide your video ID (from the URL)
video_id = "lcjdwSY2AzM"  # From https://youtu.be/lcjdwSY2AzM?si=A17IW0h6_hv2eaCj
summary = summarize_youtube_video(video_id)
print("🔍 Summary:\n", summary)


Device set to use cpu


🔍 Summary:
 Use the following pieces of context to answer the question at the end. Emmy Noether came up with a theory that reshaped our understanding of the universe. Her theories are what have gotten us the closest we've ever come to a theory of everything. But all of this and more will be covered in a second video. Make sure you're subscribed to get notified when that video comes out.
